# Import packages

---



In [ ]:
import speech_recognition as sr
from gtts import gTTS
import transformers
from transformers import pipeline
import time
import os
import datetime
import numpy as np

# Pre-pocessing and Corpus creation 

In [ ]:
import pandas as pd
df=pd.read_csv("...Flight_Timings.csv") #mention the path which contains the downloaded CSV file
df

,From,To,Departure Time,Arrival Time
0,india,india,1:00 AM,2:00AM
1,india,india,2:00 AM,3:00AM
2,Rwanda,Switzerland,2:51 AM,8:13 AM
3,United Kingdom,United States Minor Outlying Islands,9:58 AM,3:39 PM
4,United Arab Emirates,Bangladesh,6:22 PM,11:55 PM
...,...,...,...,...
236,Qatar,Bouvet Island,1:30 AM,4:49 AM
237,Mozambique,Guyana,2:49 AM,3:24 AM
238,United Arab Emirates,Liechtenstein,11:05 AM,11:25 AM
239,Cuba,Yemen,10:17 AM,12:14 PM


In [ ]:
#Converting tabular dataset into sentences
record1=""

# For a better recognition and easier pronunciation, we abbreviate some data.
for i in range (0,len(df)):
    df['From'][i]=df['From'][i].replace('United Kingdom','UK')  
    df['From'][i]=df['From'][i].replace('United Arab Emirates','UAE')
    df['To'][i]=df['To'][i].replace('United Kingdom','UK')
    df['To'][i]=df['To'][i].replace('United Arab Emirates','UAE')
    
    # Here in record1, we create the Corpus required to know the arrival time, departure time of bus/plane from onne place to another
    record1+="Bus from "+df['From'][i]+" to "+df['To'][i]+" arrives at "+df['Arrival Time'][i]+" and departs at "+df['Departure Time'][i]+". "
    
print(record1)

Bus from india to india arrives at 2:00AM and departs at    1:00 AM. Bus from india to india arrives at 3:00AM and departs at    2:00 AM. Bus from Rwanda to Switzerland arrives at 8:13 AM and departs at 2:51 AM. Bus from UK to United States Minor Outlying Islands arrives at 3:39 PM and departs at 9:58 AM. Bus from UAE to Bangladesh arrives at 11:55 PM and departs at 6:22 PM. Bus from Mexico to Australia arrives at 12:02 PM and departs at 9:06 AM. Bus from Hungary to Angola arrives at 12:20 AM and departs at 8:55 PM. Bus from Qatar to Afghanistan arrives at 1:40 PM and departs at 7:58 AM. Bus from Jamaica to Uganda arrives at 9:26 AM and departs at 2:09 AM. Bus from Holy See (Vatican City State) to Montserrat arrives at 4:29 AM and departs at 3:45 AM. Bus from Rwanda to Angola arrives at 2:42 AM and departs at 9:31 PM. Bus from Samoa to Liechtenstein arrives at 3:22 PM and departs at 9:54 AM. Bus from Kazakhstan to Bouvet Island arrives at 3:24 AM and departs at 8:44 PM. Bus from South 

In [ ]:
record2=""
record3=""
from_d={}
to_d={}

for i in range(0,len(df)):
    if(df['From'][i] not in from_d.keys()):
        from_d[df['From'][i]] = list([df['To'][i]])
    else:
        from_d[df['From'][i]].append(df['To'][i])

for i in from_d:
    txt = ""
    for j in from_d[i]:
        txt+= j+", "
    # In record2, we create a Corpus which contains no. of bus/plane from a place(destination)    
    record2+="There are "+str(len(from_d[i]))+" buses from " + i +". "
    # In record3, we create a Corpus which contains all the destinations of each bus/train
    record3+= "bus from "+i +" goes to places "+txt+". "
    
    
print(record2)
print("\n")
print(record3)

There are 2 buses from india. There are 13 buses from Rwanda. There are 4 buses from UK. There are 6 buses from UAE. There are 15 buses from Mexico. There are 9 buses from Hungary. There are 10 buses from Qatar. There are 11 buses from Jamaica. There are 11 buses from Holy See (Vatican City State). There are 10 buses from Samoa. There are 12 buses from Kazakhstan. There are 12 buses from South Georgia and the South Sandwich Islands. There are 9 buses from Congo. There are 5 buses from French Guiana. There are 10 buses from Malawi. There are 9 buses from Ethiopia. There are 8 buses from Norway. There are 10 buses from Bangladesh. There are 6 buses from Saint Kitts and Nevis. There are 13 buses from Yemen. There are 10 buses from Cuba. There are 9 buses from French Southern Territories. There are 8 buses from Chile. There are 5 buses from Palau. There are 5 buses from Tunisia. There are 8 buses from Liechtenstein. There are 6 buses from Mauritania. There are 5 buses from Mozambique. 


b

# BusBuddy Code
---



In [ ]:
class ChatBot():
    def __init__(self, name):
        print("----- Starting up", name, "-----")
        self.name = name
    def speech_to_text(self):
        recognizer = sr.Recognizer()
        with sr.Microphone() as mic:
            
            print("Listening...")
            audio = recognizer.adjust_for_ambient_noise(mic)
            audio = recognizer.listen(mic)
            self.text="ERROR"
        try:
            self.text = recognizer.recognize_google(audio)
            print("Me  --> ", self.text)
        except:
            print("Me  -->  ERROR")
    @staticmethod
    def text_to_speech(text):     
        print("Dev --> ", text)              
        speaker = gTTS(text=text, lang="en", slow=False)
        speaker.save("res.mp3")
        statbuf = os.stat("res.mp3")
        mbytes = statbuf.st_size / 1024
        duration = mbytes / 200
        os.system('start res.mp3')  #if you are using mac->afplay or else for windows->start
        # os.system("close res.mp3")
        time.sleep(int(50*duration))
        os.remove("res.mp3")
    def wake_up(self, text):
        return True if self.name in text.lower() else False
    @staticmethod
    def action_time():
        return datetime.datetime.now().time().strftime('%H:%M')

# Running BusBuddy 

---



In [ ]:
# Running the AI
if __name__ == "__main__":
    ai = ChatBot(name="dev")
    os.environ["TOKENIZERS_PARALLELISM"] = "true"
    ex=True
    while ex:
        ai.speech_to_text()
        ## wake up
        if ai.wake_up(ai.text) is True:
            res = "Hello I am Dave the AI, what can I do for you?"
        ## action time
        elif "date" and "yesterday" in ai.text:
           res = ai.action_()
        ## respond politely
        
        elif any(i in ai.text for i in ["thank","thanks"]):
            res = np.random.choice(["you're welcome!","anytime!","no problem!","cool!","I'm here if you need me!","mention not"])
        elif any(i in ai.text for i in ["exit","close"]):
            res = np.random.choice(["Tata","Have a good day","Bye","Goodbye","Hope to meet soon","peace out!"])
            ex=False
        ## conversation
        else:   
            if ai.text=="ERROR":
                res="Sorry, come again?"
            else:

                # "test-squad-trained" is a pre-trained FineTune_Distilbert model which should be downloaded from this repository "https://github.com/Raagulbharatwaj/BERT/blob/2408a6487b44fb19b5f797261b4e9f139de963ec/BERT%20For%20Question%20answering/FineTune_Distilbert.ipynb"

                model_checkpoint = "C:\\Users\\jayas\\Downloads\\test-squad-trained\\test-squad-trained" 
                question_answerer = pipeline("question-answering", model=model_checkpoint)
                question = ai.text
                
                
                if(any(i in ai.text for i in ["how many","how","many","number of buses"])):
                    reply=question_answerer(question=question, context=record2)
                    if(reply['score'] < 0.2):
                        res="Sorry, come again?"
                    else:
                        res = reply['answer'] 
                                       
                elif(any(i in ai.text for i in ["goes","go"])):
                    reply=question_answerer(question=question, context=record3)
                    if(reply['score'] < 0.0):
                        res="Sorry, come again?"
                    else:
                        res = reply['answer'] 
                               
                elif(any(i in ai.text for i in ["when","bus"])):
                    reply=question_answerer(question=question, context=record1)
                    if(reply['score'] < 0.2):
                        res="Sorry, come again?"
                    else:
                        res = reply['answer'] 
                        
        ai.text_to_speech(res)
    print("----- Closing down Dev -----")
    

----- Starting up dev -----
Listening...
Me  -->  hi Dev
Dev -->  Hello I am Dave the AI, what can I do for you?
Listening...
Me  -->  ERROR
Dev -->  Sorry, come again?
Listening...
Me  -->  when will the bus from UAE to Bangladesh arrive


C:\Users\jayas\anaconda3\lib\site-packages\numpy\core\_asarray.py:102: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return array(a, dtype, copy=False, order=order)


Dev -->  11:55 PM
Listening...
Me  -->  ERROR
Dev -->  Sorry, come again?
Listening...
Me  -->  when will the bus from Qatar to Afghanistan arrived


C:\Users\jayas\anaconda3\lib\site-packages\numpy\core\_asarray.py:102: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return array(a, dtype, copy=False, order=order)


Dev -->  1:40 PM
Listening...
Me  -->  how many buses are there from Qatar
Dev -->  10
Listening...
Me  -->  something wise
Dev -->  10
Listening...
Me  -->  close
Dev -->  Goodbye
----- Closing down Dev -----
